GAGAS Evaluation

In [18]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter 
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [94]:
# Data loader
def data_loader(file_path= '../prompts/Weekly_Challenge_Week_6.txt'):
    loader = TextLoader(file_path)
    documents = loader.load()

    # Chunk the data
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    return chunks

In [107]:
data_loader()

Created a chunk of size 1757, which is longer than the specified 500
Created a chunk of size 1320, which is longer than the specified 500
Created a chunk of size 859, which is longer than the specified 500
Created a chunk of size 517, which is longer than the specified 500


[Document(page_content='10 Academy Cohort A\nWeekly Challenge: Week 6\nPrecision RAG: Prompt Tuning For Building Enterprise Grade RAG Systems', metadata={'source': '../prompts/Weekly_Challenge_Week_6.txt'}),
 Document(page_content="Business objective  \nPromptlyTech is an innovative e-business specializing in providing AI-driven solutions for optimizing the use of Language Models (LLMs) in various industries. The company aims to revolutionize how businesses interact with LLMs, making the technology more accessible, efficient, and effective. By addressing the challenges of prompt engineering, the company plays a pivotal role in enhancing decision-making, operational efficiency, and customer experience across various industries. PromptlyTech's solutions are designed to cater to the evolving needs of a digitally-driven business landscape, where speed and accuracy are key to staying competitive.\nThe company focuses on key services: Automatic Prompt Generation, Automatic Evaluation Data Ge

In [96]:
chunks =  data_loader()

Created a chunk of size 1757, which is longer than the specified 500
Created a chunk of size 1320, which is longer than the specified 500
Created a chunk of size 859, which is longer than the specified 500
Created a chunk of size 517, which is longer than the specified 500


In [88]:
import os
 # Initialize Pinecone
pinecone.init(
    api_key=os.environ.get('7663f65c-a68e-4f31-a9a8-cf6d9e7bdac2'),
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
)

In [89]:
import time

index_name = '10-acadamy'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pinecone.Index(index_name)

In [90]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [97]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [104]:
import pandas as pd
data = pd.DataFrame(chunks)

In [109]:
from tqdm.auto import tqdm  # for progress bar
import pandas as pd

# Assuming 'chunks' is a list of Document objects as provided
data = pd.DataFrame([{'page_content': doc.page_content, 'source': doc.metadata['source']} for doc in chunks])

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i + batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{i}-{source}" for i, source in enumerate(batch['source'], start=i)]
    # get text to embed
    texts = batch['page_content'].tolist()
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [{'text': text, 'source': source} for text, source in zip(texts, batch['source'])]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

  0%|          | 0/1 [00:00<?, ?it/s]


AuthenticationError: Incorrect API key provided: sk-RtbPi***************************************BOq7. You can find your API key at https://platform.openai.com/account/api-keys.

In [75]:
import time
import os
import pinecone

def create_retriever(chunks):

  # Load Pinecone API key from .env file
  load_dotenv(find_dotenv())

 # Initialize Pinecone
  pinecone.init(
    api_key=os.environ.get('7663f65c-a68e-4f31-a9a8-cf6d9e7bdac2'),
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
  )

 # Create a new Pinecone index
  index_name = "my-index"
  pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
  while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)
 # Get the Pinecone index
  index = pinecone.Index(index_name)

 # Populate Pinecone index
  for chunk in chunks:
    embeddings = OpenAIEmbeddings().embed_query(chunk)
    index.upsert(ids=[chunk], vectors=embeddings)

 # Define Pinecone index as retriever to enable semantic search
  retriever = index.as_retriever()
  return retriever

In [76]:
chunks =  data_loader()

Created a chunk of size 1757, which is longer than the specified 500
Created a chunk of size 1320, which is longer than the specified 500
Created a chunk of size 859, which is longer than the specified 500
Created a chunk of size 517, which is longer than the specified 500


AuthenticationError: Incorrect API key provided: sk-RtbPi***************************************BOq7. You can find your API key at https://platform.openai.com/account/api-keys.

In [82]:
# Define LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define prompt template
template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use two sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

NameError: name 'retriever' is not defined